In [ ]:
%env ANYWIDGET_HMR=1

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/clicker/clicker_all_features.ipynb)

# Clicker — All Features

Comprehensive demo of every Clicker capability:
basic atom picking, custom scale/dot size, image replacement, coordinate retrieval,
lattice basis definition, and PyTorch tensor input.

## 1. Basic HAADF-STEM atom picking

Hexagonal lattice simulating a [110] zone axis. Click on bright atom columns to select positions.

In [ ]:
import numpy as np
from quantem.widget import Clicker


def make_haadf_stem(size=256, spacing=18, sigma=2.8):
    """HAADF-STEM image with atomic columns on a hexagonal lattice."""
    y, x = np.mgrid[:size, :size]
    img = np.random.normal(0.08, 0.015, (size, size))
    a1 = np.array([spacing, 0.0])
    a2 = np.array([spacing * 0.5, spacing * np.sqrt(3) / 2])
    for i in range(-1, size // spacing + 2):
        for j in range(-1, size // spacing + 2):
            cx = i * a1[0] + j * a2[0]
            cy = i * a1[1] + j * a2[1]
            if -spacing < cx < size + spacing and -spacing < cy < size + spacing:
                intensity = 0.7 + 0.3 * ((i + j) % 3 == 0)
                img += intensity * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sigma**2))
    scan_noise = np.random.normal(0, 0.01, (size, 1)) * np.ones((1, size))
    img += scan_noise
    return np.clip(img, 0, None).astype(np.float32)


haadf = make_haadf_stem()
w1 = Clicker(haadf, max_points=3)
w1

## 2. Custom scale, dot size, max points

Zoomed-in view with larger markers and more allowed selections.

In [ ]:
w2 = Clicker(haadf, scale=2.0, dot_size=18, max_points=10)
w2

## 3. Replace image with `set_image()`

Switch between two different zone axes without creating a new widget.
The cubic [001] pattern has a simple square lattice, while the hexagonal
pattern above has alternating column intensities.

In [ ]:
def make_cubic_stem(size=256, spacing=20, sigma=2.5):
    """HAADF-STEM of cubic [001] zone axis."""
    y, x = np.mgrid[:size, :size]
    img = np.random.normal(0.08, 0.015, (size, size))
    for i in range(-1, size // spacing + 2):
        for j in range(-1, size // spacing + 2):
            cx = i * spacing
            cy = j * spacing
            if -spacing < cx < size + spacing and -spacing < cy < size + spacing:
                img += 0.8 * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sigma**2))
    scan_noise = np.random.normal(0, 0.01, (size, 1)) * np.ones((1, size))
    img += scan_noise
    return np.clip(img, 0, None).astype(np.float32)


cubic = make_cubic_stem()
w3 = Clicker(haadf, scale=1.0, max_points=5)
w3

In [ ]:
# Replace the hexagonal image with the cubic [001] zone axis
w3.set_image(cubic)
print("Image replaced: now showing cubic [001] zone axis")

## 4. Retrieve selected points

After clicking on atom columns above, run this cell to print the pixel coordinates.

In [ ]:
for name, widget in [("w1 (hexagonal)", w1), ("w2 (zoomed)", w2), ("w3 (cubic)", w3)]:
    pts = widget.selected_points
    print(f"{name}: {len(pts)} point(s)")
    for i, p in enumerate(pts):
        print(f"  P{i}: x={p['x']:.1f}, y={p['y']:.1f}")
    print()

## 5. Define lattice basis from 3 points

Pick 3 atom columns on `w1` above: an origin and two nearest neighbors.
Then run this cell to compute lattice vectors **u** and **v**, plus the
angle between them.

In [ ]:
points = w1.selected_points
if len(points) < 3:
    print("Click 3 atom columns on w1 above, then re-run this cell.")
else:
    origin = np.array([points[0]["x"], points[0]["y"]])
    p1 = np.array([points[1]["x"], points[1]["y"]])
    p2 = np.array([points[2]["x"], points[2]["y"]])
    u = p1 - origin
    v = p2 - origin
    angle = np.degrees(np.arccos(
        np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    ))
    print(f"Origin: ({origin[0]:.1f}, {origin[1]:.1f})")
    print(f"u = ({u[0]:.1f}, {u[1]:.1f}), |u| = {np.linalg.norm(u):.1f} px")
    print(f"v = ({v[0]:.1f}, {v[1]:.1f}), |v| = {np.linalg.norm(v):.1f} px")
    print(f"Angle(u, v) = {angle:.1f} degrees")
    print(f"\nExpected for hexagonal: |u| ~ |v| ~ 18 px, angle ~ 60 degrees")

## 6. PyTorch tensor input

Clicker accepts both NumPy arrays and PyTorch tensors.

In [ ]:
import torch

haadf_tensor = torch.from_numpy(haadf)
print(f"Tensor shape: {haadf_tensor.shape}, dtype: {haadf_tensor.dtype}")

w4 = Clicker(haadf_tensor, scale=1.5, dot_size=14, max_points=5)
w4

## 7. Gallery mode — pick points across multiple images

Pass a list of images to pick points on each independently.
Click an unselected image to select it. Only the selected image allows point placement.

In [ ]:
# Gallery with 3 different crystal structures
hexagonal = make_haadf_stem(size=128, spacing=18)
cubic = make_cubic_stem(size=128, spacing=20)

# Ring pattern (simulated amorphous diffraction)
yy, xx = np.mgrid[:128, :128]
r = np.sqrt((xx - 64)**2 + (yy - 64)**2)
ring = (np.exp(-(r - 40)**2 / 20) + 0.5 * np.exp(-(r - 20)**2 / 10)).astype(np.float32)

w5 = Clicker(
    [hexagonal, cubic, ring],
    ncols=3,
    max_points=5,
    labels=["Hex [110]", "Cubic [001]", "Ring"],
)
w5

In [ ]:
# Per-image points
points = w5.selected_points
for i, (label, pts) in enumerate(zip(w5.labels, points)):
    print(f"{label}: {len(pts)} point(s)")
    for j, p in enumerate(pts):
        print(f"  P{j}: ({p['x']:.0f}, {p['y']:.0f})")

## 8. Gallery with torch tensors

In [ ]:
# Gallery with torch tensors
t1 = torch.from_numpy(hexagonal)
t2 = torch.from_numpy(cubic)
w6 = Clicker([t1, t2], ncols=2, max_points=4, labels=["Hex (torch)", "Cubic (torch)"])
w6